In [28]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

import time

In [13]:
# Replace 'your_website_url' with the actual URL of the website you want to scrape
url = 'https://today.yougov.com/ratings/entertainment/popularity/all-time-actors-actresses/all'
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the element containing the list of actors
    actors_container = soup.find('div', class_='rankings-entities-list-container')

    # Find all actor list items
    actor_items = actors_container.find_all('li', class_='ng-star-inserted')

    # Create lists to store data
    actor_names = []
    popularity_percentages = []
    fame_percentages = []

    # Loop through each actor item and extract information
    for actor_item in actor_items:
        # Extract the actor name
        actor_name = actor_item.find('img', class_='ng-star-inserted')['alt'] if actor_item.find('img', class_='ng-star-inserted') else None

        # Extract popularity percentage
        popularity_element = actor_item.find('span', class_='compact')
        popularity_percentage = popularity_element.text.split('%')[0] if popularity_element else None

        # Extract fame percentage
        fame_element = actor_item.find('span', class_='rankings-item-active')
        fame_percentage = fame_element.text.split('%')[0] if fame_element else None

        # Append data to lists
        actor_names.append(actor_name)
        popularity_percentages.append(popularity_percentage)
        fame_percentages.append(fame_percentage)

    # Create a DataFrame
    data = {
        'Actor Name': actor_names,
        'Popularity (%)': popularity_percentages,
        'Fame (%)': fame_percentages
    }

    df = pd.DataFrame(data)

    # Print or use the DataFrame as needed
    display(df)

else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


,Actor Name,Popularity (%),Fame (%)
0,Morgan Freeman,98,86
1,Betty White,95,83
2,Robin Williams,96,83
3,Michael J. Fox,96,81
4,Denzel Washington,97,79
5,Keanu Reeves,95,79
6,Harrison Ford,95,78
7,Samuel L. Jackson,96,78
8,Sandra Bullock,94,77
9,Danny DeVito,95,75


In [37]:
# url = 'https://today.yougov.com/ratings/entertainment/popularity/all-time-actors-actresses/all'
# path = "C:\\Users\\anejj\\Downloads\\chromedriver-win64"

# chrome_options = webdriver.ChromeOptions()
# chrome_options.binary_location = r"C:\Program Files\Google\Chrome\Application\chrome.exe"  # Add the correct path to your Chrome executable
# driver = webdriver.Chrome(executable_path=path, options=chrome_options)

# #service = Service(executable_path=path)
# driver.get(url)


url = 'https://today.yougov.com/ratings/entertainment/popularity/all-time-actors-actresses/all'
path = "C:\\Users\\anejj\\Downloads\\chromedriver-win64"

# Disable Selenium Manager
import os
os.environ["WDM_LOG_LEVEL"] = "0"

chrome_options = Options()
chrome_options.binary_location = r"C:\Program Files\Google\Chrome\Application\chrome.exe"  # Add the correct path to your Chrome executable

# Use only chrome_options, not executable_path
driver = webdriver.Chrome(options=chrome_options)

driver.get(url)


# Scroll down to load more data
while True:
    # Scroll to the bottom of the page
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)

    # Wait for a short interval to allow the content to load
    time.sleep(2)

    # Check if there are more actors loaded
    load_more_button = driver.find_element(By.XPATH, '//button[contains(text(), "Load More")]')
    if not load_more_button.is_enabled():
        break

# Get the HTML content after all data is loaded
html = driver.page_source

# Close the browser
driver.quit()

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Find the element containing the list of actors
actors_container = soup.find('div', class_='rankings-entities-list-container')

# Find all actor list items
actor_items = actors_container.find_all('li', class_='ng-star-inserted')

# Create lists to store data
actor_names = []
popularity_percentages = []
fame_percentages = []

# Loop through each actor item and extract information
for actor_item in actor_items:
    # Extract the actor name
    actor_name = actor_item.find('img', class_='ng-star-inserted')['alt'] if actor_item.find('img', class_='ng-star-inserted') else None

    # Extract popularity percentage
    popularity_element = actor_item.find('span', class_='compact')
    popularity_percentage = popularity_element.text.split('%')[0] if popularity_element else None

    # Extract fame percentage
    fame_element = actor_item.find('span', class_='rankings-item-active')
    fame_percentage = fame_element.text.split('%')[0] if fame_element else None

    # Append data to lists
    actor_names.append(actor_name)
    popularity_percentages.append(popularity_percentage)
    fame_percentages.append(fame_percentage)

# Create a DataFrame
data = {
    'Actor Name': actor_names,
    'Popularity Percentage': popularity_percentages,
    'Fame Percentage': fame_percentages
}

df = pd.DataFrame(data)

# Print or use the DataFrame as needed
print(df)


Unable to obtain driver using Selenium Manager: c:\Users\anejj\anaconda3\lib\site-packages\selenium\webdriver\common\windows\selenium-manager.exe is missing.  Please open an issue on https://github.com/SeleniumHQ/selenium/issues


SeleniumManagerException: Message: c:\Users\anejj\anaconda3\lib\site-packages\selenium\webdriver\common\windows\selenium-manager.exe is missing.  Please open an issue on https://github.com/SeleniumHQ/selenium/issues
